In [ ]:
from data_processing import *
import datetime 
import pandas as pd
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', 100) 


In [ ]:
data = read_data('countypres_2000-2020.csv')
df = pd.DataFrame(data)
df['state'] = df['state'].apply(lambda x: "District of Columbia" if x == "DISTRICT OF COLUMBIA" else x.title())
df['county_name'] = df['county_name'].apply(lambda x: "District of Columbia" if x == "DISTRICT OF COLUMBIA" else x.title())
df.set_index(['state', 'county_name', 'year'], inplace=True)
additional_data = []
for loc_idx in list(set(df.index)):
    iloc_idx = np.array(df.loc[loc_idx]['candidatevotes'].apply(int)).argmax()
    dict = {}
    name = df.loc[loc_idx].iloc[iloc_idx].name
    dict['State'] = name[0]
    dict['Year'] = name[2]
    dict['County'] = name[1]
    dict['Party'] = 'D' if df.loc[loc_idx].iloc[iloc_idx].to_dict()['party'] == 'DEMOCRAT' else 'R'
    additional_data += [dict]


In [ ]:
data = read_data('by_county.csv')
data = data + additional_data
data = sorted(data, key=lambda d: d['State'])
data = sorted(data, key=lambda d: int(d['Year']))

filename_t1 = datetime.datetime.now().__str__()
with open(filename_t1, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['State', 'Year', 'County', 'Party'])
    writer.writeheader()
    writer.writerows(data)


In [ ]:


# for _ in data:
   
#     _['State'] = _['state']
#     del _['state']
#     _['State'] = _['State'].capitalize()

#     _['Year'] = _['year']
#     del _['year']

#     _['Party'] = _['party']
#     del _['party']
#     _['Party'] = 'D' if _['Party'] =='DEMOCRAT' else 'R'

#     _['County'] = _['county_name']
#     del _['county_name']
#     _['County'] = _['County'].capitalize()
    

#     del _['candidate']
#     del _['candidatevotes']
#     del _['county_fips']
#     del _['version']
#     del _['mode']
#     del _['totalvotes']
#     del _['office']
#     del _['state_po']




In [ ]:
'LA', 'AK', 'WA'

In [ ]:
WA_result = []
for i in range(10):
    try:
        year = 2000 - 4*i
        http = "https://en.wikipedia.org/wiki/" + str(year) + "_United_States_presidential_election_in_Washington_(state)"
        f = r.get(http)
        html_content = f.content
        soup = BeautifulSoup(html_content, 'html.parser')
        tables = [table for table in soup.find_all("table")]
        # states_list = []
        # for _ in table.find_all("tr")[2:]:
        #     states_list += [_.find("td").text]
        # return states_list[:-2]
        for _ in tables[11].find_all("tr")[2:]:
            try:
                dict = {}
                county = _.find("td").text.strip('\n')
                party = "R" if _.find("td")['style'].strip('background-color:') == '#FFB6B6' else "D"
                
                dict['State'] = 'Washington'
                dict['Year'] = year
                dict['County'] = county
                dict['Party'] = party
                WA_result += [dict]
            except Exception as e:
                print(e)
                pass
    except Exception as e:
        print(e)
        pass

In [ ]:
WA_result

In [ ]:
LA_result = []
for i in range(10):
    try:
        year = 2000 - 4*i
        http = "https://en.wikipedia.org/wiki/" + str(year) + "_United_States_presidential_election_in_Louisiana"
        f = r.get(http)
        html_content = f.content
        soup = BeautifulSoup(html_content, 'html.parser')
        tables = [table for table in soup.find_all("table")]
        # states_list = []
        # for _ in table.find_all("tr")[2:]:
        #     states_list += [_.find("td").text]
        # return states_list[:-2]
        for _ in tables[12].find_all("tr")[2:]:
            try:
                dict = {}
                county = _.find("td").text.strip('\n')
                party = "R" if _.find("td")['style'].strip('background-color:') == '#FFB6B6' else "D"
                
                dict['State'] = 'Louisiana'
                dict['Year'] = year
                dict['County'] = county
                dict['Party'] = party
                LA_result += [dict]
            except Exception as e:
                print(e)
                pass
    except Exception as e:
        print(e)
        pass
LA_result

In [ ]:
# AK_result = []
# for i in range(10):
#     try:
#         year = 2000 - 4*i
#         http = "https://en.wikipedia.org/wiki/" + str(year) + "_United_States_presidential_election_in_Louisiana"
#         f = r.get(http)
#         html_content = f.content
#         soup = BeautifulSoup(html_content, 'html.parser')
#         tables = [table for table in soup.find_all("table")]
#         # states_list = []
#         # for _ in table.find_all("tr")[2:]:
#         #     states_list += [_.find("td").text]
#         # return states_list[:-2]
#         for _ in tables[12].find_all("tr")[2:]:
#             try:
#                 dict = {}
#                 county = _.find("td").text.strip('\n')
#                 party = "R" if _.find("td")['style'].strip('background-color:') == '#FFB6B6' else "D"
                
#                 dict['State'] = 'Louisiana'
#                 dict['Year'] = year
#                 dict['County'] = county
#                 dict['Party'] = party
#                 AK_result += [dict]
#             except Exception as e:
#                 print(e)
#                 pass
#     except Exception as e:
#         print(e)
#         pass
# AK_result

#Alaska has no county-level election results, pass.


In [ ]:
from importlib import reload
import data_processing
reload(data_processing)
data = data_processing.read_data(filename_t1)
data = data + LA_result + WA_result

data = sorted(data, key=lambda d: d['State'])
data = sorted(data, key=lambda d: int(d['Year']))
filename_t2 = datetime.datetime.now().__str__()
data_processing.write_data(filename_t2, data)

In [ ]:
reload(data_processing)
raw_data = read_data(filename_t2)
results = raw_data

# MERGING CAINC DATA WITH BY COUNTY ELECTION RESULTS.
for result in results:
    if int(result['Year']) <= 1972:
        pass
    try:
        state_abbrev = misc.us_state_to_abbrev[result["State"]]
        with open('CAINC30_Economic Profile by County/CAINC30_'+ state_abbrev + '_1969_2023.csv', 'r') as file:
            csv_reader = csv.DictReader(file)
            data = list(csv_reader)[:-5]
        for _ in data:
            del _["Region"]
            del _["TableName"]
            del _["Unit"]
            del _["LineCode"]
        
        
        econ_data = list(filter(lambda d: result["County"] in d["GeoName"], data))
        
        for ed in econ_data:
            year = int(result["Year"])
            label = ed["Description"]
            result[label + "YR-1"] = int(ed[str(year - 1)]) 
            result[label + "YR-2"] = int(ed[str(year - 2)])
            result[label + "YR-3"] = int(ed[str(year - 3)])
            result[label + "YR-4"] = int(ed[str(year - 4)])
            result['GeoFIPS'] = ed['GeoFIPS']

    except Exception as e:
        print(e)
        pass
pd.DataFrame(results)

In [ ]:
filename_t3 = datetime.datetime.now().__str__()
pd.DataFrame(results).to_csv(filename_t3)

In [ ]:

for result in results:
    state_abbrev = misc.us_state_to_abbrev[result["State"]]
    try:
        with open('CAINC1_Annual Personal Income by County/CAINC1_'+ state_abbrev + '_1969_2023.csv', 'r') as file:
            csv_reader = csv.DictReader(file)
            data = list(csv_reader)[:-5]
        for _ in data:
            del _["Region"]
            del _["TableName"]
            del _["Unit"]
            del _["LineCode"]
        
        
        econ_data = list(filter(lambda d: result["County"] in d["GeoName"], data))
        
        for ed in econ_data:
            year = int(result["Year"])
            label = ed["Description"]
            result[label + "YR-1"] = int(ed[str(year - 1)]) 
            result[label + "YR-2"] = int(ed[str(year - 2)])
            result[label + "YR-3"] = int(ed[str(year - 3)])
            result[label + "YR-4"] = int(ed[str(year - 4)])
    except Exception as e:
        print(e)
        pass

filename_t4 = datetime.datetime.now().__str__()
pd.DataFrame(results).to_csv(filename_t4)



In [ ]:
# REMOVE ROWS WITH TOO MANY MISSING DATA
df = pd.DataFrame(results)
columns = df.columns
data = list(filter(lambda x: len(x.values()) == len(columns), results))


In [ ]:

# ADJUST FOR INFLATION USING PANDAS
df = pd.DataFrame(data)
# remove population columns
columns = df.columns
populations = [df['Population (persons) 1/YR-1'],
            df['Population (persons) 1/YR-2'],
            df['Population (persons) 1/YR-3'], 
            df['Population (persons) 1/YR-4']]
fips = df['GeoFIPS']
df.drop(columns = "Population (persons) 1/YR-1", inplace=True)
df.drop(columns = "Population (persons) 1/YR-2", inplace=True)
df.drop(columns = "Population (persons) 1/YR-3", inplace=True)
df.drop(columns = "Population (persons) 1/YR-4", inplace=True)
df.drop(columns = " Population (persons) 3/YR-1", inplace=True)
df.drop(columns = " Population (persons) 3/YR-2", inplace=True)
df.drop(columns = " Population (persons) 3/YR-3", inplace=True)
df.drop(columns = " Population (persons) 3/YR-4", inplace=True)
df.drop(columns = "GeoFIPS", inplace=True)
#adjust for inflation
for i in df.index:
    try:
        year = int(df.iloc[i,1])
        df.iloc[i, 4:] = df.iloc[i, 4:].map(lambda x: int(x) * misc.cpi_dict[year])
    except Exception as e:
        print(e)
        pass

#add population columns back
df["Population YR-1"] = populations[0]
df["Population YR-2"] = populations[1]
df["Population YR-3"] = populations[2]
df["Population YR-4"] = populations[3]
df['GeoFIPS'] = fips

# NORMALIZE DATA
for column in df.columns:
    try:
        if (column != "Year") and (column!='GeoFIPS'):
            df[column] = df[column].map(lambda x: int(x))
            x_max = max(list(df[column]))
            x_min = min(list(df[column]))
            df[column] = df[column].map(lambda x: (x - x_min)/(x_max - x_min))
    except Exception as e:
        print(e)
filename = datetime.datetime.now().__str__()
df.to_csv(filename)